In [ ]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    AdamW
)
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from typing import Dict
from sklearn.preprocessing import MultiLabelBinarizer

# Configuration for all models
MODEL_CONFIGS = {

    "deberta-v3": {
        "name": "microsoft/deberta-v3-base",
        "quantize": False,
        "max_length": 512,
        "batch_size": 16
    }
}

class PropagandaDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return loss.mean()

def load_data(file_path: str) -> list:
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

def prepare_datasets(config: Dict, tokenizer, train_texts, train_labels, dev_texts, dev_labels):
    def tokenize(texts):
        return tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=config["max_length"],
            return_tensors="pt"
        )

    train_encodings = tokenize(train_texts)
    dev_encodings = tokenize(dev_texts)

    return (
        PropagandaDataset(train_encodings, train_labels),
        PropagandaDataset(dev_encodings, dev_labels)
    )

def train_model(model_config: Dict, train_data, dev_data, all_labels):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    if model_config["quantize"]:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        model = AutoModelForSequenceClassification.from_pretrained(
            model_config["name"],
            num_labels=len(all_labels),
            quantization_config=bnb_config,
            device_map="auto",
            problem_type="multi_label_classification"
        )
    else:  # For non-quantized models like DeBERTa
        model = AutoModelForSequenceClassification.from_pretrained(
            model_config["name"],
            num_labels=len(all_labels),
            problem_type="multi_label_classification"
        ).to(device)  # Manually move to device


    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_config["name"])
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    # Prepare data
    train_dataset, dev_dataset = prepare_datasets(
        model_config, tokenizer, *train_data, *dev_data
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=model_config["batch_size"],
        shuffle=Truea
    )

    dev_loader = DataLoader(
        dev_dataset,
        batch_size=model_config["batch_size"],
        shuffle=False
    )

    # Initialize training components
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
    criterion = FocalLoss(
    alpha=1.0,
    gamma=2.0
)

    # Training loop
    best_f1 = 0
    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Evaluation
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in dev_loader:
                inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                labels = batch['labels'].cpu().numpy()

                outputs = model(**inputs).logits
                preds = torch.sigmoid(outputs).cpu().numpy()

                all_preds.append(preds)
                all_labels.append(labels)

        # Calculate metrics

        preds = np.concatenate(all_preds)
        labels = np.concatenate(all_labels)

        # Threshold optimization
        thresholds = np.linspace(0.3, 0.7, 20)
        best_thresh = 0.5
        best_f1 = 0

        # Find optimal threshold
        y_true = labels
        for thresh in thresholds:
            current_f1 = f1_score(y_true, (preds > thresh).astype(int),
                                average='micro', zero_division=0)
            if current_f1 > best_f1:
                best_f1 = current_f1
                best_thresh = thresh

        # Apply best threshold
        preds_binary = (preds > best_thresh).astype(int)


        # Compute metrics
        f1_micro = f1_score(labels, preds_binary, average='micro')
        f1_macro = f1_score(labels, preds_binary, average='macro')
        precision = precision_score(labels, preds_binary, average='micro')
        recall = recall_score(labels, preds_binary, average='micro')

        # Print metrics
        print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")
        print(f"  F1-Micro: {f1_micro:.4f} | F1-Macro: {f1_macro:.4f}")
        print(f"  Precision: {precision:.4f} | Recall: {recall:.4f}")

        # Save best model
        if f1_micro > best_f1:
            best_f1 = f1_micro
            torch.save(model.state_dict(), f"best_{model_config['name'].replace('/', '_')}.pth")

    return best_f1

from google.colab import drive
drive.mount('/content/drive')

data_dir = "/content/drive/My Drive/SEMEVAL/data/"

# Load data
def load_data(file_path: str) -> list:
    full_path = data_dir + file_path
    with open(full_path, "r", encoding="utf-8") as f:
        return json.load(f)


# Model configuration
NUM_EPOCHS = 15


# === Main Execution ===
MODEL_TO_TRAIN = "deberta-v3"

# Load data
train_data = load_data("training_set_task1.txt")
dev_data = load_data("dev_set_task1.txt")

train_texts = [item["text"] for item in train_data]
train_labels = [item["labels"] for item in train_data]
dev_texts = [item["text"] for item in dev_data]
dev_labels = [item["labels"] for item in dev_data]

# Encode labels
all_labels = sorted({label for labels in train_labels + dev_labels for label in labels})
mlb = MultiLabelBinarizer(classes=all_labels)
train_labels_enc = mlb.fit_transform(train_labels)
dev_labels_enc = mlb.transform(dev_labels)

# Run training
if MODEL_TO_TRAIN == 'all':
    results = {}
    for model_name, config in MODEL_CONFIGS.items():
        print(f"\n=== Training {model_name} ===")
        score = train_model(config, (train_texts, train_labels_enc), (dev_texts, dev_labels_enc), all_labels)
        results[model_name] = score
    print("\nTraining results:", results)
else:
    if MODEL_TO_TRAIN not in MODEL_CONFIGS:
        raise ValueError(f"Unknown model: {MODEL_TO_TRAIN}. Available: {list(MODEL_CONFIGS.keys())}")
    print(f"\n=== Training {MODEL_TO_TRAIN} ===")
    score = train_model(MODEL_CONFIGS[MODEL_TO_TRAIN], (train_texts, train_labels_enc), (dev_texts, dev_labels_enc), all_labels)
    print(f"\nFinal {MODEL_TO_TRAIN} F1: {score:.4f}")

Mounted at /content/drive

=== Training deberta-v3 ===


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 13 | Loss: 0.0359
  F1-Micro: 0.6018 | F1-Macro: 0.1869
  Precision: 0.6602 | Recall: 0.5528


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 14 | Loss: 0.0334
  F1-Micro: 0.5616 | F1-Macro: 0.1972
  Precision: 0.7125 | Recall: 0.4634
Epoch 15 | Loss: 0.0312
  F1-Micro: 0.6000 | F1-Macro: 0.2189
  Precision: 0.5906 | Recall: 0.6098

Final deberta-v3 F1: 0.6000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
